In [1]:
import os
import pandas as pd

# Path to the directory containing the CSV files
directory_path = '/data1-8tb-blk/sp2023stock/Insight-Wave/TopicModeling/result/integrationOutput/id/kaohoon_ids.csv'

articles_list = {}

df = pd.read_csv(directory_path)
articles_list = df['ID'].tolist()

print(f'max = {max(articles_list)}, min ={min(articles_list)}')

max = 646621, min =578616


In [2]:
import os
main = os.getcwd()
dataset_kaohoon = main + '/../../../data/kaohoon/article/'

start_process = main +  '/../../../src/Parsing/kaohoon/parsing_progress.txt'
end_process = main + '/../../../src/Crawling/kaohoon/progress.txt'

In [3]:
import os
import csv
import re
import pandas as pd
import chardet
import datetime
from bs4 import BeautifulSoup

def htmlInput(file_path):
    with open(file_path, "r", encoding='utf-8') as file:
        html_content = file.read()
    soup = BeautifulSoup(html_content, "html.parser")
    return soup


#Extract title
def extract_title(soup):
    title = soup.title.string.strip() if soup.title else None
    return title

#Extract body
def extract_body(soup):
    article_divs = soup.find_all("div", class_="entry-content entry clearfix")
    articles = []
    if article_divs:
        for article_div in article_divs:
            articleses = article_div.find_all('p')
            for article in articleses:
                articles.append(" ".join(article.get_text().split()))
    
    return articles

#Extract tags
def extract_tags(soup):
    tagcloud_span = soup.find('span', class_='tagcloud')
    if tagcloud_span is not None:
        tags = tagcloud_span.find_all('a')
        tag_content = [tag.get_text(strip=True) for tag in tags]
        unnecessary_tag = soup.find('div', class_='post-bottom-meta post-bottom-tags post-tags-classic')
        unnecessary_tag.decompose()
    else:
        tag_content = ['Tag not found']
    return tag_content

#Extract date
def extract_pubdate(soup):
    date_div = soup.find('div', {'id': 'single-post-meta'})
    date_span = date_div.find('span', {'class': 'date meta-item tie-icon'})
    timestamps = date_span.text.strip() if date_span else None
    if timestamps is not None:
        date_obj = datetime.datetime.strptime(timestamps, '%d/%m/%Y')
        pubdate = date_obj.strftime('%Y-%m-%d')
    else:
        pubdate = None
    return pubdate

#Extract title
def extract_intro(soup):
    intro_div = soup.find("h2", class_="entry-sub-title")
    intro = ""
    if intro_div:
        intro = intro_div.get_text(strip=True)
    else:
        intro = None

    return intro

def extract_urlPic(soup):
    picture_div = soup.find('figure', class_="single-featured-image")
    if picture_div:
        img_tag = picture_div.find('img')
        if img_tag:
            url = img_tag.get('src')
            print(url)
        else:
            print("No img tag found inside the figure")
    else:
        print("No figure with class 'single-featured-image' found")
    
    return url

# #Start
# if os.path.exists(start_process):
#     with open('parsing_progress.txt','r') as f:
#         progress_start = int(f.readline().strip())
#         start_id = progress_start - 1
# else: 
#     print("Start path doesn't exists process start from 1")
#     start_id = 1 
    
# article_id = start_id

# #End
# if os.path.exists(end_process):
#     with open(process,'r') as f:
#         progress_end = int(f.readline().strip())
#         end_id = progress_end + 1
# else: 
#     print("End path doesn't exists")
# print(f'Start parsing at {start_id} and Ending at {end_id}' )


print(f'Start parsing process')
for article_id in articles_list: #range(start_id, end_id+1):
    file_path = dataset_kaohoon + str(article_id) + "/index.txt"
    print(f'Processing article id {article_id}')
    if os.path.exists(file_path):
        try:
            soup = htmlInput(file_path)
            title = extract_title(soup)
            intro = extract_intro(soup)
            articles = extract_body(soup)
            pubdate = extract_pubdate(soup)
            tag_content = extract_tags(soup)
            url = extract_urlPic(soup)

            data_dict = {'Title': title, 'Intro': intro, 'Article': articles, 'DateTime': pubdate, 'Tags': tag_content, 'url_picture': url}

            output_path = dataset_kaohoon+ str(article_id) + "/parsing.txt"
            with open(os.path.join(output_path), 'w', encoding="utf-8") as f:
                for key, value in data_dict.items():
                    if value is not None:
                        if key == 'Tags' and isinstance(value, list):
                            f.write(f"[::{key}::]\n")
                            for tags in value:
                                f.write(f"{tags}\n")

                        elif key == 'Article' and isinstance(value, list):
                            f.write(f"[::{key}::]\n")
                            for bodys in value:
                                if bodys == 'SPONSORED':
                                    continue
                                else:
                                    f.write(f"{bodys}\n")
                                    f.write("\n")
                        else:
                            f.write(f"[::{key}::]\n{value}\n")

                            
            with open('parsing_progress.txt', 'w') as f:
                f.write(f'{article_id}')

            with open('parsing_number.csv', 'a', newline='') as f:
                writer = csv.writer(f)
                validation = 'Valid'
                writer.writerow([article_id, validation])

                    
        except Exception as e:
            print(f"Error processing article {article_id}: {e}")
            with open('parsing_number.csv', 'a', newline='') as f:
                writer = csv.writer(f)
                validation = 'Error'
                writer.writerow([article_id, validation])
                continue

print(f'Saving')

# # Write the last processed article_id to 'csv_progress.txt' after loop ends
# with open('parsing_progress.txt', 'w') as f:
#     f.write(str(article_id) + '\n')

print(f'finished')

Start parsing process
Processing article id 646621
https://www.kaohoon.com/wp-content/uploads/2023/12/IMG_3478.jpeg
Processing article id 646609
https://www.kaohoon.com/wp-content/uploads/2021/09/2021-09-03_ดัชนี-PMI-ภาคบริการจีน.jpg
Processing article id 646607
https://www.kaohoon.com/wp-content/uploads/2020/09/Down.jpg
Processing article id 646603
https://www.kaohoon.com/wp-content/uploads/2023/01/ราชกิจจานุเบกษา.jpg
Processing article id 646599
https://www.kaohoon.com/wp-content/uploads/2023/12/2023-12-30_ผลสลาก.jpg
Processing article id 646593
https://www.kaohoon.com/wp-content/uploads/2023/01/AOT_2023-01-18_ประกาศผลการสรรหาผู้อำนวยการใหญ่-AOT.jpg
Processing article id 646590
https://www.kaohoon.com/wp-content/uploads/2023/12/2023-12-30_แอร์พอร์ตลิงก์ขยายเวลาถึงตี2.jpg
Processing article id 646588
https://www.kaohoon.com/wp-content/uploads/2023/12/2023-12-30_เตือนรถโดยสาร-อย่าฉวยโอกาสขึ้นราคาช่.jpg
Processing article id 646586
https://www.kaohoon.com/wp-content/uploads/2023/02/PM2.